In [1]:
import numpy as np
import pandas as pd
import os
from function.function_date import vectorized_ROCtoAD, vectorized_date_tran, vectorized_thirdWednesday,\
                                  vectorized_betwween_days
top_path = './../../'
Data_path = top_path + 'Data/'
Organized_path = Data_path + 'Organized'
if 'Organized' not in os.listdir(Data_path):
    os.mkdir(Organized_path)
NoIV_path = Organized_path + '/NoIV'
IV_path = Organized_path + '/IV'
year_path = Organized_path + '/year'

if 'NoIV' not in os.listdir(Organized_path):
    os.mkdir(NoIV_path)
if 'IV' not in os.listdir(Organized_path):
    os.mkdir(IV_path)
if 'year' not in os.listdir(Organized_path):
    os.mkdir(year_path)

整理利率資料：讀取出五家銀行個期別歷史利率資料並取平均

In [2]:
start_date = '10101'
stop_date = '11301'
start_year = 2012
end_year = 2023


#整理利率資料：讀取出五家銀行個期別歷史利率資料並取平均
month_num =  (int(stop_date[:3])-1 - (int(start_date[:3])+1) +1)*12 \
    + int(stop_date[3:]) + (12-int(start_date[3:])+1)

blanks = ['台銀', '土銀', '合庫', '一銀', '華銀']
sub_matrix = np.zeros((month_num, 8))
RateMatrix = np.tile(sub_matrix, (len(blanks), 1, 1))

for i in range(len(blanks)):
    rate_data = pd.read_excel(Data_path+'UnOrganized/rate/a13rate.xls', sheet_name= blanks[i])
    rate_data = rate_data.iloc[:, [0,2, 6, 8, 10, 12, 14, 16, 18]]
    
    rate_data = rate_data[4:]
    rate_matrix = np.array(rate_data)
    idx = np.where(rate_matrix[:,0] == start_date )[0][0]
    rate_matrix = rate_matrix[idx:]
    
    RateMatrix[i] = rate_matrix[:,1:]
date_matrix = np.reshape(rate_matrix[:,0], (-1, 1))
date_matrix = vectorized_ROCtoAD(date_matrix)
avg_rate_matrix = RateMatrix.mean(axis=0)
avg_rate_matrix = avg_rate_matrix/100
avg_rate_matrix = np.hstack((date_matrix, avg_rate_matrix))
column_names = ['日期', '活期', '一個月', '三個月', '六個月', \
                '九個月', '一年','二年', '三年']
avg_rate_data = pd.DataFrame(data=avg_rate_matrix, columns=column_names)
avg_rate_data.to_csv(NoIV_path+'/5BAvgRate.csv', encoding='Big5', index=False)

從期貨資料中篩出契約TX的資料，接著篩出盤內交易的資料，找出結算價為0的資料，代表交易日當日就是最後交易日。  
如果是要找出今年的最後交易日，先自己算出各月份的第三個禮拜三，再去台灣期貨交易所的官網找行事曆比較，把不一樣的月份的日期再改掉即可。

In [3]:
years = range(start_year , end_year + 1, 1)
this_year = 2024


LastTrandingDays = pd.DataFrame({
    '交易日期':[],
    '到期月份(週別)':[]
})
for year in years:
    fut_data = pd.read_csv(Data_path+'UnOrganized/{}/fut/{}_fut.csv'.format(year, year), \
                       encoding='Big5', index_col=False)

    TX_data  = fut_data[fut_data['契約'] == 'TX']
    #TX_data = ful_data[ful_data.iloc[:, 1] == 'TX']

    TX_data = TX_data[TX_data['結算價'] != '-']
    TX_data = TX_data[TX_data['收盤價'] != '-']
    TX_data['結算價'] = TX_data['結算價'].astype(int)
    TX_data['收盤價'] = TX_data['收盤價'].astype(int)      
    TX_data['到期月份(週別)'] = TX_data['到期月份(週別)'].astype(str)
    TX_data['到期月份(週別)'] = TX_data['到期月份(週別)'].str.strip()
    TX_data = TX_data[TX_data['到期月份(週別)'].str.len() == 6]
    lastTrandingDays = TX_data[TX_data['結算價'] == 0]
    lastTrandingDays = lastTrandingDays[['交易日期', '到期月份(週別)']]
    LastTrandingDays = pd.concat([LastTrandingDays, lastTrandingDays], axis=0)

LTD_matrix = np.array(LastTrandingDays)
ThisYearMonth = np.array(['{}0{}'.format(this_year, i+1) if i < 9 \
                 else '{}{}'.format(this_year, i+1) for i in range(12)] )
ThisYearDate = vectorized_thirdWednesday(ThisYearMonth)

ThisYearDate[0] =  '2024/01/17' 
ThisYear = np.hstack((np.reshape(ThisYearDate, (-1, 1)), np.reshape(ThisYearMonth, (-1, 1))))

LTD_matrix = np.vstack((LTD_matrix, ThisYear))
LTD_matrix[:, 0] = vectorized_date_tran(LTD_matrix[:, 0])
LastTrandingDays = pd.DataFrame(data=LTD_matrix, columns=['交易日期', '到期月份(週別)'])
LastTrandingDays.to_csv('./../../Data/Organized/NoIV/LastTrandingDays.csv',  encoding='Big5', index=False)

C:\Users\bgpa4\AppData\Local\Temp\ipykernel_21576\648897484.py:10: DtypeWarning: Columns (2,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  fut_data = pd.read_csv(Data_path+'UnOrganized/{}/fut/{}_fut.csv'.format(year, year), \
C:\Users\bgpa4\AppData\Local\Temp\ipykernel_21576\648897484.py:10: DtypeWarning: Columns (2,3,4,5,6,9,10,11,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  fut_data = pd.read_csv(Data_path+'UnOrganized/{}/fut/{}_fut.csv'.format(year, year), \
C:\Users\bgpa4\AppData\Local\Temp\ipykernel_21576\648897484.py:10: DtypeWarning: Columns (3,4,5,6,9,10,11,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  fut_data = pd.read_csv(Data_path+'UnOrganized/{}/fut/{}_fut.csv'.format(year, year), \
C:\Users\bgpa4\AppData\Local\Temp\ipykernel_21576\648897484.py:10: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  fut_data = 

在Data/Organized/year/下，建立各年份的資料夾

In [4]:
import os 
for year in range(start_year, end_year+1):
    if str(year)   not in os.listdir(year_path ):
        os.mkdir(year_path +'/{}'.format(year))

整理各年份的TX與TXO資料，擷取出交易日期、到期月份(週別)、履約價、買賣權、成交量、收盤價、期貨成交量、期貨收盤價的資料，匯出為csv檔案，分別儲存到各年份的資料夾

In [8]:
import os 
from function.function_select import  select_index
years = range(start_year, end_year+1, 1)
TXO_columns = ['交易日期', '到期月份(週別)', '履約價', '買賣權', '成交量','收盤價', '結算價','期貨成交量', \
                    '期貨開盤價', '期貨最高價', '期貨最低價','期貨收盤價', '期貨結算價']

for year in years:
    ful_data = pd.read_csv(Data_path+'UnOrganized/{}/fut/{}_fut.csv'.format(year, year, year), \
                        encoding='Big5', index_col=False)

    TX_data  = ful_data[ful_data['契約'] == 'TX']
    #TX_data = ful_data[ful_data.iloc[:, 1] == 'TX']
    TX_data = TX_data[TX_data['結算價'] != '-']
    TX_data = TX_data[TX_data['收盤價'] != '-']
    TX_data = TX_data[TX_data['開盤價'] != '-']
    TX_data = TX_data[TX_data['最高價'] != '-']
    TX_data = TX_data[TX_data['最低價'] != '-']
    
    TX_data['收盤價'] = TX_data['收盤價'].astype(int)   
    TX_data['成交量'] = TX_data['成交量'].astype(int) 
    TX_data['最高價'] = TX_data['最高價'].astype(int) 
    TX_data['最低價'] = TX_data['最低價'].astype(int) 
    TX_data['開盤價'] = TX_data['開盤價'].astype(int) 
    TX_data['結算價'] = TX_data['結算價'].astype(int) 
    TX_data['到期月份(週別)'] = TX_data['到期月份(週別)'].astype(str)
    TX_data['到期月份(週別)'] = TX_data['到期月份(週別)'].str.strip()
    TX_data = TX_data[['交易日期', '到期月份(週別)', '成交量', '開盤價', '最高價', \
                       '最低價', '收盤價', '結算價']]


    opt_files = os.listdir(Data_path +'UnOrganized/{}/opt/'.format(year))
    TXO_data_year = pd.DataFrame(data=[], columns=TXO_columns[:7])
    for opt_file in opt_files:
        option_data = pd.read_csv(Data_path+'UnOrganized/{}/opt/{}'.format(year, opt_file), \
                              encoding='Big5', index_col=False)
        TXO_data_month = option_data[option_data['契約'] == 'TXO']
        TXO_data_month = TXO_data_month[TXO_data_month['結算價'] != '-']
        TXO_data_month = TXO_data_month[TXO_data_month['收盤價'] != '-']
        TXO_data_month = TXO_data_month[TXO_columns[:7]]
        TXO_data_month['到期月份(週別)'] = TXO_data_month['到期月份(週別)'].astype(str)
        #TXO_data.iloc[:, 1] = TXO_data.iloc[:, 1].astype(str)
        TXO_data_month['到期月份(週別)'] = TXO_data_month['到期月份(週別)'].str.strip()
        #TXO_data.iloc[:, 1] = TXO_data.iloc[:, 1].str.strip()
        TXO_data_month = TXO_data_month[TXO_data_month['到期月份(週別)'].str.len() == 6]
        #TXO_data = TXO_data[TXO_data.iloc[:, 1].str.len() == 6]
        TXO_data_month.loc[TXO_data_month['買賣權'] == '買權', '買賣權'] = 'call'
        TXO_data_month.loc[TXO_data_month['買賣權'] == '賣權', '買賣權'] = 'put'
        #TXO_data.loc[TXO_data.iloc[:, 3] == '買權', '買賣權'] = 'call'
        #TXO_data.loc[TXO_data.iloc[:, 3] == '賣權', '買賣權'] = 'put'
        TXO_data_year = pd.concat([TXO_data_year, TXO_data_month], axis=0)
    TX_matrix = np.array(TX_data)
    TXO_matrix = np.array(TXO_data_year)

    idx_TX_match, idx_TXO_match = select_index(TX_matrix[:, :2], TXO_matrix[:, :2], axis=1) 
    TXO_matrix = np.hstack((TXO_matrix[idx_TXO_match], TX_matrix[idx_TX_match, 2:]))      

    TXO_matrix[:, 0] = vectorized_date_tran(TXO_matrix[:, 0])
    
    
    
    TXO_data = pd.DataFrame(data=TXO_matrix, columns=TXO_columns)
    TXO_data.to_csv(year_path +'/{}/TXO_{}.csv'.format(year, year),  encoding='Big5', index=False)


C:\Users\bgpa4\AppData\Local\Temp\ipykernel_21576\4122930854.py:8: DtypeWarning: Columns (2,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  ful_data = pd.read_csv(Data_path+'UnOrganized/{}/fut/{}_fut.csv'.format(year, year, year), \
C:\Users\bgpa4\AppData\Local\Temp\ipykernel_21576\4122930854.py:34: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  option_data = pd.read_csv(Data_path+'UnOrganized/{}/opt/{}'.format(year, opt_file), \
C:\Users\bgpa4\AppData\Local\Temp\ipykernel_21576\4122930854.py:8: DtypeWarning: Columns (2,3,4,5,6,9,10,11,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  ful_data = pd.read_csv(Data_path+'UnOrganized/{}/fut/{}_fut.csv'.format(year, year, year), \
C:\Users\bgpa4\AppData\Local\Temp\ipykernel_21576\4122930854.py:8: DtypeWarning: Columns (3,4,5,6,9,10,11,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  ful

讀取存取好的各年度的TXO資料，合併成一個好幾年分的TXO資料並匯出csv檔案

In [9]:
TXO_data_all  = pd.DataFrame(data=[], columns=TXO_columns)
for year in range(start_year, end_year+1, 1):
    TXO_data_year = pd.read_csv(year_path + '/{}/TXO_{}.csv'.format(year, year), \
                                 encoding='Big5', index_col=False)
    TXO_data_all = pd.concat([TXO_data_all, TXO_data_year])
TXO_data_all.to_csv(NoIV_path + '/TXO_all.csv',  encoding='Big5', index=False)

讀取TXO_all與全部的LastTrandingDays的資料，根據TXO_all中每列的到期月份(週別)欄位的資料，找到與LastTrandingDays中到期月份(週別)欄位資料相同的那一列，並把那列的交易日期的資料取代TXO_all到期月份(週別)的資料，最後TXO_all的到期月份(週別)的資料就變成到期日期的資料。並計算交易日期與到期日期的天數差距，創造一個到期天數的新的欄位。  
讀取5BAvgRate的資料，根據每列TXO_all的資料，從5BAvgRate中找到與交易日期同月的資料，並根據TXO_all到期天數的資料，找到5BAvgRate中匹配天期利率的資料，並把這些資料取出來，作為TXO_all新的欄位：無風險利率的資料，最後再匯出為TXO_LTDR.csv。  



In [10]:

TXO_data_all = pd.read_csv(NoIV_path + '/TXO_all.csv',  encoding='Big5', index_col=False)
LTD_data = pd.read_csv(NoIV_path + '/LastTrandingDays.csv',  encoding='Big5', index_col=False)

TXO_matrix = np.array(TXO_data_all)
LTD_matrix = np.array(LTD_data)

new_column = np.empty(len(TXO_data_all), dtype=object)


for i in range(len(TXO_matrix)):
    match_condition = np.all(TXO_matrix[i, 1:2] == LTD_matrix[:, 1:2], axis=1)
    TXO_matrix[i][1] = LTD_matrix[match_condition][0][0]


T_array = vectorized_betwween_days(TXO_matrix[:,0], TXO_matrix[:,1])
TXO_matrix = np.hstack((TXO_matrix, np.reshape(T_array, (-1, 1))))


rate_data = pd.read_csv(NoIV_path + '/5BAvgRate.csv',  encoding='Big5', index_col=False)
avg_rate_matrix = np.array(rate_data)
new_column = np.empty(TXO_matrix.shape[0], dtype=object)
for i in range(TXO_matrix.shape[0]):
    date = TXO_matrix[i, 0:1][0][0:7]
    match_condition = np.all(date == avg_rate_matrix[:, 0:1] , axis=1)
    k = i
    for k in range(i, -1,-1):
        date = TXO_matrix[k, 0:1][0][0:7]
        match_condition = np.all(date == avg_rate_matrix[:, 0:1], axis =1)
        if np.any(match_condition):
            break

    if TXO_matrix[k, -1] < 28:
        new_column[i] = avg_rate_matrix[match_condition, 1][0]
    elif TXO_matrix[k, -1] < 89:
        new_column[i] = avg_rate_matrix[match_condition, 2][0]
    elif TXO_matrix[k, -1] < 181:
        new_column[i] = avg_rate_matrix[match_condition, 3][0]
    elif TXO_matrix[k, -1] < 270:
        new_column[i] = avg_rate_matrix[match_condition, 4][0]
    elif TXO_matrix[k, -1] < 365:
        new_column[i] = avg_rate_matrix[match_condition, 5][0]
    else:
        new_column[i] = avg_rate_matrix[match_condition, 6][0]
TXO_matrix = np.hstack((TXO_matrix, np.reshape(new_column, (-1, 1))))


TXO_columns.extend(['到期天數', '無風險利率'])
TXO_columns[1] = '到期日期'
TXO_data_new = pd.DataFrame(data=TXO_matrix, columns=TXO_columns)
TXO_data_new.to_csv(NoIV_path + '/TXO_LTDR.csv',  encoding='Big5', index=False)